In [86]:
import sagemaker
import boto3
import sys
import pandas as pd
import numpy as np 
import io 
from sagemaker.session import Session
from sagemaker import get_execution_role
from time import gmtime,strftime,sleep
from sagemaker.feature_store.feature_group import FeatureGroup


In [87]:
prefix="feature_store_aws"
role=get_execution_role()
sagemaker_session=Session() 
region=sagemaker_session.boto_region_name
s3_bucket_name="mybacket05"

In [88]:
print(f"Region is : {region}")
print(f"Role is : {role}")

Region is : us-east-1
Role is : arn:aws:iam::640168443051:role/service-role/AmazonSageMaker-ExecutionRole-20250523T211400


In [89]:
customer_data=pd.read_csv("customers.csv")
orders_data=pd.read_csv("orders.csv")

In [90]:
customer_data.head()

,Unnamed: 0,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,0,C1,0,1,2024-05-02T05:39:10.965Z,0,0,0,0,0,1,0.203425
1,1,C2,1,1,2024-05-02T05:39:10.966Z,0,0,0,0,0,1,0.859589
2,2,C3,1,1,2024-05-02T05:39:10.967Z,0,1,0,0,0,0,0.527397
3,3,C4,1,0,2024-05-02T05:39:10.967Z,0,0,0,0,1,0,0.780822
4,4,C5,1,0,2024-05-02T05:39:10.968Z,0,0,0,0,1,0,0.691096


In [91]:
customer_data.drop(columns=["Unnamed: 0"],axis=1,inplace=True)

In [92]:
customer_data.head()

,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,C1,0,1,2024-05-02T05:39:10.965Z,0,0,0,0,0,1,0.203425
1,C2,1,1,2024-05-02T05:39:10.966Z,0,0,0,0,0,1,0.859589
2,C3,1,1,2024-05-02T05:39:10.967Z,0,1,0,0,0,0,0.527397
3,C4,1,0,2024-05-02T05:39:10.967Z,0,0,0,0,1,0,0.780822
4,C5,1,0,2024-05-02T05:39:10.968Z,0,0,0,0,1,0,0.691096


In [93]:
orders_data.head()

,order_id,customer_id,product_id,purchase_amount,is_reordered,event_time,n_days_since_last_purchase
0,O1,C9765,P11660,0.572673,0,2024-05-02T05:39:17.172Z,0.273256
1,O2,C3674,P6868,0.693861,0,2024-05-02T05:39:17.172Z,0.846899
2,O3,C2139,P4749,0.556139,1,2024-05-02T05:39:17.172Z,0.408915
3,O4,C7794,P542,0.043069,1,2024-05-02T05:39:17.172Z,0.843023
4,O5,C2229,P7605,0.463861,1,2024-05-02T05:39:17.172Z,0.265504


# Create feature group

In [94]:
customers_feature_group_name = "customers-fg-"+strftime("%d-%H-%M-%S", gmtime()) 
orders_feature_group_name = "orders-fg-"+strftime("%d-%H-%M-%S", gmtime()) 

In [95]:
customer_feature_group=FeatureGroup(name=customers_feature_group_name,
                                   sagemaker_session=sagemaker_session)
order_feature_group=FeatureGroup(name=orders_feature_group_name,sagemaker_session=sagemaker_session)

In [96]:
record_identifier_feature_name = "customer_id"

# Load Feature definitions

In [97]:
customer_feature_group.load_feature_definitions(data_frame = customer_data)

[FeatureDefinition(feature_name='customer_id', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='sex', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='is_married', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='event_time', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='age_18-29', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='age_30-39', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='age_40-49', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='age_50-59', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='age_60-69

# Load Feature definitions

In [98]:
order_feature_group.load_feature_definitions(data_frame = orders_data)

[FeatureDefinition(feature_name='order_id', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='customer_id', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='product_id', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='purchase_amount', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='is_reordered', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='event_time', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='n_days_since_last_purchase', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None)]

In [99]:
customer_feature_group.create(s3_uri = f"s3://{s3_bucket_name}/{prefix}",
                               record_identifier_name=record_identifier_feature_name, event_time_feature_name="event_time",
                                role_arn=role,
                                enable_online_store=True)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:640168443051:feature-group/customers-fg-31-15-07-28',
 'ResponseMetadata': {'RequestId': 'd3f71e4d-c150-42f3-876a-6bcb66e07896',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd3f71e4d-c150-42f3-876a-6bcb66e07896',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '101',
   'date': 'Sat, 31 May 2025 15:07:29 GMT'},
  'RetryAttempts': 0}}

In [100]:
order_feature_group.create(s3_uri = f"s3://{s3_bucket_name}/{prefix}",
                               record_identifier_name=record_identifier_feature_name, event_time_feature_name="event_time",
                                role_arn=role,
                                enable_online_store=True)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:640168443051:feature-group/orders-fg-31-15-07-28',
 'ResponseMetadata': {'RequestId': '8d01f156-eb79-419f-8290-f859cb9bdce8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8d01f156-eb79-419f-8290-f859cb9bdce8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '98',
   'date': 'Sat, 31 May 2025 15:07:30 GMT'},
  'RetryAttempts': 1}}

In [101]:
customer_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:640168443051:feature-group/customers-fg-31-15-07-28',
 'FeatureGroupName': 'customers-fg-31-15-07-28',
 'RecordIdentifierFeatureName': 'customer_id',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'customer_id',
   'FeatureType': 'String'},
  {'FeatureName': 'sex', 'FeatureType': 'Integral'},
  {'FeatureName': 'is_married', 'FeatureType': 'Integral'},
  {'FeatureName': 'event_time', 'FeatureType': 'String'},
  {'FeatureName': 'age_18-29', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_30-39', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_40-49', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_50-59', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_60-69', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_70-plus', 'FeatureType': 'Integral'},
  {'FeatureName': 'n_days_active', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2025, 5, 31, 15, 7, 28, 601000, tzinfo=tzlocal()),

In [102]:
order_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:640168443051:feature-group/orders-fg-31-15-07-28',
 'FeatureGroupName': 'orders-fg-31-15-07-28',
 'RecordIdentifierFeatureName': 'customer_id',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'order_id', 'FeatureType': 'String'},
  {'FeatureName': 'customer_id', 'FeatureType': 'String'},
  {'FeatureName': 'product_id', 'FeatureType': 'String'},
  {'FeatureName': 'purchase_amount', 'FeatureType': 'Fractional'},
  {'FeatureName': 'is_reordered', 'FeatureType': 'Integral'},
  {'FeatureName': 'event_time', 'FeatureType': 'String'},
  {'FeatureName': 'n_days_since_last_purchase', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2025, 5, 31, 15, 7, 30, 320000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': 's3://mybacket05/feature_store_aws',
   'ResolvedOutputS3Uri': 's3://mybacket05/feature_store_aws/640168443051/sagema

# Add metadata to a feature
# description and parameters

In [104]:
from sagemaker.feature_store.inputs import FeatureParameter

customer_feature_group.update_feature_metadata(feature_name = "customer_id", description= "The ID of the customer, it is also part of Order feature group",
                                                parameter_additions=[FeatureParameter("idType","primarykey")])

{'ResponseMetadata': {'RequestId': 'c0229e72-d149-4a6e-8cc7-2a2c382648b1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c0229e72-d149-4a6e-8cc7-2a2c382648b1',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 31 May 2025 15:07:55 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [105]:
customer_feature_group.describe_feature_metadata("customer_id")

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:640168443051:feature-group/customers-fg-31-15-07-28',
 'FeatureGroupName': 'customers-fg-31-15-07-28',
 'FeatureName': 'customer_id',
 'FeatureType': 'String',
 'CreationTime': datetime.datetime(2025, 5, 31, 15, 7, 28, 601000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 5, 31, 15, 7, 55, 137000, tzinfo=tzlocal()),
 'Description': 'The ID of the customer, it is also part of Order feature group',
 'Parameters': [{'Key': 'idType', 'Value': 'primarykey'}],
 'ResponseMetadata': {'RequestId': 'd5a79eab-5e08-47e5-b782-dfce4eece072',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd5a79eab-5e08-47e5-b782-dfce4eece072',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '398',
   'date': 'Sat, 31 May 2025 15:07:58 GMT'},
  'RetryAttempts': 0}}

In [106]:
sagemaker_session.boto_session.client("sagemaker", region_name=region).search(
    Resource="FeatureMetadata",
    SearchExpression={
        "Filters":[
            {
                "Name": "FeatureGroupName",
                "Operator": "Contains",
                "Value": "customers-fg"
            },
            {"Name": "Parameters.idType", "Operator": "Equals", "Value": "primarykey"}
        ]
    })

{'Results': [{'FeatureMetadata': {'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:640168443051:feature-group/customers-fg-31-15-07-28',
    'FeatureGroupName': 'customers-fg-31-15-07-28',
    'FeatureName': 'customer_id',
    'FeatureType': 'String',
    'CreationTime': datetime.datetime(2025, 5, 31, 15, 7, 28, tzinfo=tzlocal()),
    'LastModifiedTime': datetime.datetime(2025, 5, 31, 15, 7, 55, tzinfo=tzlocal()),
    'Description': 'The ID of the customer, it is also part of Order feature group',
    'Parameters': [{'Key': 'idType', 'Value': 'primarykey'}]}},
  {'FeatureMetadata': {'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:640168443051:feature-group/customers-fg-31-14-44-16',
    'FeatureGroupName': 'customers-fg-31-14-44-16',
    'FeatureName': 'customer_id',
    'FeatureType': 'String',
    'CreationTime': datetime.datetime(2025, 5, 31, 14, 44, 16, tzinfo=tzlocal()),
    'LastModifiedTime': datetime.datetime(2025, 5, 31, 14, 44, 41, tzinfo=tzlocal()),
    'Description': 'The ID

# Ingest the data to the Feature Group

In [107]:
customer_feature_group.ingest(data_frame = customer_data, max_workers=3,wait=True)

IngestionManagerPandas(feature_group_name='customers-fg-31-15-07-28', feature_definitions={'customer_id': {'FeatureName': 'customer_id', 'FeatureType': 'String'}, 'sex': {'FeatureName': 'sex', 'FeatureType': 'Integral'}, 'is_married': {'FeatureName': 'is_married', 'FeatureType': 'Integral'}, 'event_time': {'FeatureName': 'event_time', 'FeatureType': 'String'}, 'age_18-29': {'FeatureName': 'age_18-29', 'FeatureType': 'Integral'}, 'age_30-39': {'FeatureName': 'age_30-39', 'FeatureType': 'Integral'}, 'age_40-49': {'FeatureName': 'age_40-49', 'FeatureType': 'Integral'}, 'age_50-59': {'FeatureName': 'age_50-59', 'FeatureType': 'Integral'}, 'age_60-69': {'FeatureName': 'age_60-69', 'FeatureType': 'Integral'}, 'age_70-plus': {'FeatureName': 'age_70-plus', 'FeatureType': 'Integral'}, 'n_days_active': {'FeatureName': 'n_days_active', 'FeatureType': 'Fractional'}}, sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f00b0648a70>, sagemaker_session=<sagemaker.session.Session o

In [108]:
order_feature_group.ingest(data_frame = orders_data, max_workers=3,wait=True)

IngestionManagerPandas(feature_group_name='orders-fg-31-15-07-28', feature_definitions={'order_id': {'FeatureName': 'order_id', 'FeatureType': 'String'}, 'customer_id': {'FeatureName': 'customer_id', 'FeatureType': 'String'}, 'product_id': {'FeatureName': 'product_id', 'FeatureType': 'String'}, 'purchase_amount': {'FeatureName': 'purchase_amount', 'FeatureType': 'Fractional'}, 'is_reordered': {'FeatureName': 'is_reordered', 'FeatureType': 'Integral'}, 'event_time': {'FeatureName': 'event_time', 'FeatureType': 'String'}, 'n_days_since_last_purchase': {'FeatureName': 'n_days_since_last_purchase', 'FeatureType': 'Fractional'}}, sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f00b0648a70>, sagemaker_session=<sagemaker.session.Session object at 0x7f00bcabc440>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7f00bc31ff50>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

In [109]:
customer_id = "C1"
sample_record = sagemaker_session.boto_session.client(
    "sagemaker-featurestore-runtime", region_name=region
).get_record(
    FeatureGroupName=customers_feature_group_name, RecordIdentifierValueAsString=str(customer_id)
)

In [110]:
sample_record

{'ResponseMetadata': {'RequestId': '24298d37-de1a-4a62-967b-879fdc13f70c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '24298d37-de1a-4a62-967b-879fdc13f70c',
   'content-type': 'application/json',
   'content-length': '877',
   'date': 'Sat, 31 May 2025 15:19:51 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'customer_id', 'ValueAsString': 'C1'},
  {'FeatureName': 'sex', 'ValueAsString': '0'},
  {'FeatureName': 'is_married', 'ValueAsString': '1'},
  {'FeatureName': 'event_time', 'ValueAsString': '2024-05-02T05:39:10.965Z'},
  {'FeatureName': 'age_18-29', 'ValueAsString': '0'},
  {'FeatureName': 'age_30-39', 'ValueAsString': '0'},
  {'FeatureName': 'age_40-49', 'ValueAsString': '0'},
  {'FeatureName': 'age_50-59', 'ValueAsString': '0'},
  {'FeatureName': 'age_60-69', 'ValueAsString': '0'},
  {'FeatureName': 'age_70-plus', 'ValueAsString': '1'},
  {'FeatureName': 'n_days_active', 'ValueAsString': '0.2034246575342466'}]}

In [111]:
all_records = sagemaker_session.boto_session.client(
    "sagemaker-featurestore-runtime", region_name=region
).batch_get_record(
    Identifiers=[
        {
            "FeatureGroupName": customers_feature_group_name,
            "RecordIdentifiersValueAsString": ["C1","C2"],
        },
        {
            "FeatureGroupName": orders_feature_group_name,
            "RecordIdentifiersValueAsString": ["C1","C2"],
        },
    ]
)

In [112]:
all_records

{'ResponseMetadata': {'RequestId': '8d4ad708-3241-4e8b-acd5-e5358b702fdd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8d4ad708-3241-4e8b-acd5-e5358b702fdd',
   'content-type': 'application/json',
   'content-length': '3408',
   'date': 'Sat, 31 May 2025 15:19:57 GMT'},
  'RetryAttempts': 0},
 'Records': [{'FeatureGroupName': 'customers-fg-31-15-07-28',
   'RecordIdentifierValueAsString': 'C1',
   'Record': [{'FeatureName': 'customer_id', 'ValueAsString': 'C1'},
    {'FeatureName': 'sex', 'ValueAsString': '0'},
    {'FeatureName': 'is_married', 'ValueAsString': '1'},
    {'FeatureName': 'event_time', 'ValueAsString': '2024-05-02T05:39:10.965Z'},
    {'FeatureName': 'age_18-29', 'ValueAsString': '0'},
    {'FeatureName': 'age_30-39', 'ValueAsString': '0'},
    {'FeatureName': 'age_40-49', 'ValueAsString': '0'},
    {'FeatureName': 'age_50-59', 'ValueAsString': '0'},
    {'FeatureName': 'age_60-69', 'ValueAsString': '0'},
    {'FeatureName': 'age_70-plus', 'ValueAsStr

In [113]:
from sagemaker.feature_store.feature_definition import StringFeatureDefinition

customer_feature_group.update(feature_additions=[StringFeatureDefinition("email")])

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:640168443051:feature-group/customers-fg-31-15-07-28',
 'ResponseMetadata': {'RequestId': '8742ecec-e3ba-4098-949d-9665049cc405',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8742ecec-e3ba-4098-949d-9665049cc405',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '101',
   'date': 'Sat, 31 May 2025 15:20:03 GMT'},
  'RetryAttempts': 0}}

In [114]:
customer_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:640168443051:feature-group/customers-fg-31-15-07-28',
 'FeatureGroupName': 'customers-fg-31-15-07-28',
 'RecordIdentifierFeatureName': 'customer_id',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'customer_id',
   'FeatureType': 'String'},
  {'FeatureName': 'sex', 'FeatureType': 'Integral'},
  {'FeatureName': 'is_married', 'FeatureType': 'Integral'},
  {'FeatureName': 'event_time', 'FeatureType': 'String'},
  {'FeatureName': 'age_18-29', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_30-39', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_40-49', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_50-59', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_60-69', 'FeatureType': 'Integral'},
  {'FeatureName': 'age_70-plus', 'FeatureType': 'Integral'},
  {'FeatureName': 'n_days_active', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2025, 5, 31, 15, 7, 28, 601000, tzinfo=tzlocal()),

# Updated Customers Data

In [115]:
df_up=pd.read_csv("https://raw.githubusercontent.com/manifoldailearning/mlops-with-aws-datascientists/refs/heads/main/Section-13.1-Feature-Store/transformed/customers_updated.csv")

In [116]:
df_up.head()

,customer_id,sex,is_married,event_time,email,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,C10001,0,1,2024-05-02T13:43:28.295Z,kennethday@example.net,0,0,0,0,0,1,0.345890
1,C10002,0,1,2024-05-02T13:43:28.296Z,smelendez@example.org,0,0,0,0,0,1,0.884247
2,C10003,1,0,2024-05-02T13:43:28.298Z,brianperry@example.net,0,1,0,0,0,0,0.734247
3,C10004,0,0,2024-05-02T13:43:28.300Z,alyssamorgan@example.net,0,0,0,0,1,0,0.386301
4,C10005,0,1,2024-05-02T13:43:28.301Z,cjenkins@example.org,0,0,0,0,1,0,0.363699


In [117]:
df_up.drop(columns=["email"],axis=1,inplace=True)

In [119]:
df_up.head()

,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,C10001,0,1,2024-05-02T13:43:28.295Z,0,0,0,0,0,1,0.345890
1,C10002,0,1,2024-05-02T13:43:28.296Z,0,0,0,0,0,1,0.884247
2,C10003,1,0,2024-05-02T13:43:28.298Z,0,1,0,0,0,0,0.734247
3,C10004,0,0,2024-05-02T13:43:28.300Z,0,0,0,0,1,0,0.386301
4,C10005,0,1,2024-05-02T13:43:28.301Z,0,0,0,0,1,0,0.363699


In [118]:
customer_data.head()

,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,C1,0,1,2024-05-02T05:39:10.965Z,0,0,0,0,0,1,0.203425
1,C2,1,1,2024-05-02T05:39:10.966Z,0,0,0,0,0,1,0.859589
2,C3,1,1,2024-05-02T05:39:10.967Z,0,1,0,0,0,0,0.527397
3,C4,1,0,2024-05-02T05:39:10.967Z,0,0,0,0,1,0,0.780822
4,C5,1,0,2024-05-02T05:39:10.968Z,0,0,0,0,1,0,0.691096


In [120]:
customer_feature_group.ingest(data_frame = df_up, max_workers=3,wait=True)

IngestionManagerPandas(feature_group_name='customers-fg-31-15-07-28', feature_definitions={'customer_id': {'FeatureName': 'customer_id', 'FeatureType': 'String'}, 'sex': {'FeatureName': 'sex', 'FeatureType': 'Integral'}, 'is_married': {'FeatureName': 'is_married', 'FeatureType': 'Integral'}, 'event_time': {'FeatureName': 'event_time', 'FeatureType': 'String'}, 'age_18-29': {'FeatureName': 'age_18-29', 'FeatureType': 'Integral'}, 'age_30-39': {'FeatureName': 'age_30-39', 'FeatureType': 'Integral'}, 'age_40-49': {'FeatureName': 'age_40-49', 'FeatureType': 'Integral'}, 'age_50-59': {'FeatureName': 'age_50-59', 'FeatureType': 'Integral'}, 'age_60-69': {'FeatureName': 'age_60-69', 'FeatureType': 'Integral'}, 'age_70-plus': {'FeatureName': 'age_70-plus', 'FeatureType': 'Integral'}, 'n_days_active': {'FeatureName': 'n_days_active', 'FeatureType': 'Fractional'}, 'email': {'FeatureName': 'email', 'FeatureType': 'String'}}, sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7

In [ ]:
customer_id = "C10001"
sample_record = sagemaker_session.boto_session.client("sagemaker-featurestore-runtime", region_name=region
).get_record(
    FeatureGroupName=customer_feature_group_name, RecordIdentifierValueAsString=str(customer_id)
)